# Tarjeta de datos 1

### Hipótesis 1: "Las provincias con una edad media menor y una renta media por hogar mayor a la nacional, suelen estar más concienciadas con el uso de energías renovables y utilizan más dispositivos que aprovechan este tipo de energía."

La justificación de esta hipótesis es que las personas jóvenes suelen estar más concienciadas con el medio ambiente, además de que tienen más tiempo para amortizar la inversión de este tipo de tecnología a lo largo de su vida.
<br>
<br>

### Preparación del entorno

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm  
import numpy as np
from ydata_profiling import ProfileReport
import os 

SILVER_DATA_PATH = os.path.join("..", "data/silver/")
GOLD_DATA_PATH = os.path.join("..", "data/gold/")

### Datos

Primero, cargamos los datasets que utilizaremos en esta tarjeta, los cuales han sido generados a partir del procesamiento inicial de los datos y corresponden a la capa silver.

In [ ]:
# Distribución de edad
distribucion_edad_df=pd.read_csv(SILVER_DATA_PATH+"distribucion_edad.csv", sep = ";", encoding = "latin")
distribucion_edad_df.info()

In [ ]:
# Renta media
renta_media_hogar_df = pd.read_csv(SILVER_DATA_PATH+"renta_media_hogar.csv", sep = ";", encoding = "latin")
renta_media_hogar_df.info()

In [ ]:
# Dispositivos de energía renovable
dispositivos_renovable_df=pd.read_csv(SILVER_DATA_PATH+"dispositivos_renovable.csv", sep = ";", encoding = "latin")
dispositivos_renovable_df.info()

### Procesamiento

#### distribucion_edad_df

Del dataset distribucion_edad_df.csv queremos obtener la información sobre los intervalos de edad de cada año por provincia, además de:

- El total nacional del número de habitantes.
- La edad media por provincia.
- El número de habitantes por provincia.

In [ ]:
# Ejemplo de los datos que presenta la tabla
distribucion_edad_df.head(5)

Total nacional del número de habitantes.

In [ ]:
# Calcular la suma de la columna 'Total'
poblacion_total_nacional = distribucion_edad_df['Total'].sum()
print(f"Hay un total de {poblacion_total_nacional} habitantes en España.")

Edad media por provincia

In [ ]:
# Función para obtener la edad media de cada grupo quinquenal
def obtener_edad_media(grupo):
    # Comprobar si el grupo es "100 y más años"
    if "100 y más años" in grupo:
        return 100  # Asignamos 100 años como la edad media para este grupo
    else:
        # Extraer los valores del rango (e.g., "De 0 a 4 años" -> 0, 4)
        partes = grupo.split(' a ')
        edad_inferior = int(partes[0].split(' ')[1])
        edad_superior = int(partes[1].split(' ')[0])
        # Calcular la edad media del grupo
        return (edad_inferior + edad_superior) / 2
    
# Crear un contenedor para almacenar los resultados de edad media por provincia
edad_media_por_provincia = []
# Agrupamos los datos por provincia
for provincia, grupo in distribucion_edad_df.groupby('Provincias'):
    edad_media_nivel_provincia = 0
    poblacion_total_provincia = 0
    
    # Calculamos la edad media ponderada para cada grupo quinquenal en la provincia
    for index, row in grupo.iterrows():
        rango_edad = row['Edad (grupos quinquenales)']
        media_rango = obtener_edad_media(rango_edad)
        
        # Sumar la edad media ponderada por la población
        edad_media_ponderada = media_rango * row['Total']
        edad_media_nivel_provincia += edad_media_ponderada
        poblacion_total_provincia += row['Total']
    
    # Calcular la edad media real de la provincia
    edad_media_provincia = edad_media_nivel_provincia / poblacion_total_provincia
    edad_media_por_provincia.append({'Provincias': provincia, 'Edad media': edad_media_provincia})

# Crear un DataFrame con las provincias y su edad media
df_edad_media_provincia = pd.DataFrame(edad_media_por_provincia)

# Mostrar el DataFrame con la edad media por provincia
print(df_edad_media_provincia)



Número de habitantes por provincia

In [ ]:
# Agrupar por 'Provincias' y calcular el sumatorio de la columna 'Total'
poblacion_total_provincias = distribucion_edad_df.groupby('Provincias')['Total'].sum().reset_index()
poblacion_total_provincias.head(5)


Se modifica el nombre de la columna "Total" para mejorar su comprensión.

In [ ]:
poblacion_total_provincias = poblacion_total_provincias.rename(columns={'Total': 'Población Total'})

<br>
<br>

#### renta_media_hogar_df 

El objetivo con el dataset renta_media_hogar_df es conocer la renta media de cada provincia.

In [ ]:
# Ejemplo de los datos que presenta la tabla
renta_media_hogar_df.head(5)

In [ ]:
# Nos quedamos con los datos del 2022
renta_media_hogar_df = renta_media_hogar_df[renta_media_hogar_df['Periodo'] == 2022]
renta_media_hogar_df = renta_media_hogar_df.reset_index(drop=True)

# Eliminamos la columna 'Periodo'
renta_media_hogar_df = renta_media_hogar_df.drop('Periodo', axis=1)

renta_media_hogar_df.head(5)

Calculamos la renta media nacional.

In [ ]:
renta_media_nacional = renta_media_hogar_df['Total'].mean()

print(f"La renta media nacional por hogar es de {renta_media_nacional} euros.")


Se modifica el nombre de la columna "Total" para mejorar su comprensión.

In [ ]:
renta_media_hogar_df = renta_media_hogar_df.rename(columns={'Total': 'Renta media por hogar'})

<br>
<br>


#### dispositivos_renovable_df

In [ ]:
# Ejemplo de los datos que presenta la tabla
dispositivos_renovable_df.head(5)

Queremos generar un dataset que indique el porcentaje de la provincia que utiliza dispositivos de energía renovable

In [ ]:
# Agrupar por 'Provincias' y 'Dispone de dispositivo de energía renovable', sumando el total
dispositivos_renovable_df = dispositivos_renovable_df.groupby(['Provincias', 'Dispone de dispositivo de energía renovable'])['Total'].sum().unstack(fill_value=0)
dispositivos_renovable_df.head(5)

In [ ]:
# Se calcula el porcentaje de hogares con dispositivos de energía renovable y se eliminan las columnas "Si" y "No"
dispositivos_renovable_df['Porcentaje de hogares con dispositivos de energía renovable'] = (dispositivos_renovable_df['Si'] / (dispositivos_renovable_df['Si'] + dispositivos_renovable_df['No']))*100
dispositivos_renovable_df.drop(columns=['No', 'Si'], inplace=True)
dispositivos_renovable_df.head(5)

In [ ]:
# Las columnas están estructuradas como un MultiIndex, el objetivo es "aplanarlas" y dejar el dataset como | Provincias | Proporción ... |
dispositivos_renovable_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in dispositivos_renovable_df.columns]
dispositivos_renovable_df.reset_index(inplace=True)

dispositivos_renovable_df.head(5)

<br>


### Resultado

In [ ]:
# Unir DataFrames
data_card_1_df = pd.merge(poblacion_total_provincias, renta_media_hogar_df, on='Provincias')
data_card_1_df = pd.merge(data_card_1_df, df_edad_media_provincia, on='Provincias')
data_card_1_df = pd.merge(data_card_1_df, dispositivos_renovable_df, on='Provincias')


data_card_1_df.head(5)

Datos importantes

In [ ]:
print(f"Hay un total de {poblacion_total_nacional} habitantes en España.")
print(f"La renta media nacional por hogar es de {renta_media_nacional} euros.")

<br>

### Data Profiling

Estudio sobre población total de España.

In [ ]:
df = pd.DataFrame(data_card_1_df)
df = df.sort_values(by="Población Total", ascending=False)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
plt.bar(df["Provincias"], df["Población Total"], color='skyblue')

# Agregar títulos y etiquetas
plt.title("Población por provincia", fontsize=20)
plt.xlabel("Provincias", fontsize=12)
plt.ylabel("Población", fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotar etiquetas si son largas

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Estudio sobre la renta media por hogar en cada provincia.

In [ ]:
df = pd.DataFrame(data_card_1_df)
df = df.sort_values(by="Renta media por hogar", ascending=False)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
plt.bar(df["Provincias"], df["Renta media por hogar"], color='skyblue')

# Agregar títulos y etiquetas
plt.title("Renta media por hogar", fontsize=20)
plt.xlabel("Provincias", fontsize=12)
plt.ylabel("Renta", fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotar etiquetas si son largas

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Estudio sobre la edad media en cada provincia.

In [ ]:
df = pd.DataFrame(data_card_1_df)
df = df.sort_values(by="Edad media", ascending=False)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
plt.bar(df["Provincias"], df["Edad media"], color='skyblue')

# Agregar títulos y etiquetas
plt.title("Edad media", fontsize=20)
plt.xlabel("Provincias", fontsize=12)
plt.ylabel("Edad", fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotar etiquetas si son largas

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Estudio sobre la proporción de hogares con dispositivos de energía renovable.

In [ ]:
df = pd.DataFrame(data_card_1_df)
df = df.sort_values(by="Porcentaje de hogares con dispositivos de energía renovable", ascending=False)

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Tamaño del gráfico
plt.bar(df["Provincias"], df["Porcentaje de hogares con dispositivos de energía renovable"], color='skyblue')

# Agregar títulos y etiquetas
plt.title("Porcentaje de hogares con dispositivos de energía renovable", fontsize=20)
plt.xlabel("Provincias", fontsize=12)
plt.ylabel("%", fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotar etiquetas si son largas

# Mostrar el gráfico
plt.tight_layout()
plt.show()

Generamos un profile report del estudio.

In [ ]:
profile = ProfileReport(data_card_1_df, title="Data Card 1 - Profile", explorative=True)
profile.to_file("../profiling/data_card_1_profile_report.html")

<br>

### Generar dataset

In [ ]:
with open(GOLD_DATA_PATH + "data_card_1_df.csv", "w") as f:
    data_card_1_df.to_csv(f, sep=';', encoding='utf-8', index=False)